# Black Scholes Exercise 1: Naive implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [12]:
# Boilerplate for the example

import cProfile
import pstats

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = [0.0 for i in range(nopt)]
put = [-1.0 for i in range(nopt)]
price=list(price)
strike=list(strike)
t=list(t)

# The Naive Black Scholes algorithm (looped)

In [13]:
from math import log, sqrt, exp, erf
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes(nopt, price, strike, t, rate, vol, call, put):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = float( price [i] )
        S = strike [i]
        T = t [i]
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        call [i] = P * d1 - Se * d2
        put [i] = call [i] - P + Se

# Timeit and CProfile Tests

What do you notice about the times?

%timeit function(args)

%prun function(args)

In [14]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

212 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)')

         600004 function calls in 0.376 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.019    0.000    0.028    0.000 <ipython-input-13-2dcd212cd4c8>:2(<lambda>)
        1    0.267    0.267    0.376    0.376 <ipython-input-13-2dcd212cd4c8>:4(black_scholes)
        1    0.000    0.000    0.376    0.376 <string>:1(<module>)
        1    0.000    0.000    0.376    0.376 {built-in method builtins.exec}
   200000    0.050    0.000    0.050    0.000 {built-in method math.erf}
   100000    0.012    0.000    0.012    0.000 {built-in method math.exp}
   100000    0.019    0.000    0.019    0.000 {built-in method math.log}
   100000    0.008    0.000    0.008    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [15]:
%prun black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

         600004 function calls in 0.307 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.214    0.214    0.307    0.307 <ipython-input-13-2dcd212cd4c8>:4(black_scholes)
   200000    0.046    0.000    0.046    0.000 {built-in method math.erf}
   100000    0.016    0.000    0.023    0.000 <ipython-input-13-2dcd212cd4c8>:2(<lambda>)
   100000    0.015    0.000    0.015    0.000 {built-in method math.log}
   100000    0.009    0.000    0.009    0.000 {built-in method math.exp}
   100000    0.007    0.000    0.007    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.307    0.307 {built-in method builtins.exec}
        1    0.000    0.000    0.307    0.307 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

# Line_Profiler tests

How many times does the function items get called (hits)?

In [3]:
%load_ext line_profiler

%lprun -f function function(args)

In [19]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

Timer unit: 1e-06 s

Total time: 1.05934 s
File: <ipython-input-13-2dcd212cd4c8>
Function: black_scholes at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def black_scholes(nopt, price, strike, t, rate, vol, call, put):
     5         1         17.0     17.0      0.0      mr = -rate
     6         1         10.0     10.0      0.0      sig_sig_two = vol * vol * 2
     7                                               
     8    100001      49642.0      0.5      4.7      for i in range(nopt):
     9    100000      70868.0      0.7      6.7          P = float( price [i] )
    10    100000      50343.0      0.5      4.8          S = strike [i]
    11    100000      48071.0      0.5      4.5          T = t [i]
    12                                                   
    13    100000      70619.0      0.7      6.7          a = log(P / S)
    14    100000      60490.0      0.6      5.7          b = T * mr
    15          